#**Data Normalization (part 1)**

#**Normalizing & Scaling data**

This lesson is about preparing your data so that machine learning algorithms can be more accurate and efficient. We will discuss various techniques that all fall under the categories of data standardization, cleaning, scaling, etc. We'll use the umbrella term normalization. Although even that term means something different when talking about vector normalization.

The overall goal of data normalization is about reducing redundancy and improving accuracy and integrity of the data and for any techniques that will use the data. The process of normalizing has its roots in relational databases where the goal is to restructure the tables and relationships to get them into 'normal' form to reduce data redundancy.

#**Different Data Types, different techniques**


![](https://drive.google.com/uc?export=view&id=1UvAkd0P4H_lLomxN3qQwsHOpE5VzcWRV)

You learned about the different ways to classify data in the previous class; this lesson is looking at data at more granular level. Although all data is essentially grouped binary values (bits on or off), we will look at how to manage some of the most common categories in data- science including text, numeric, and categorical (a mix of text and numbers).

Data normalization can be done just about any kind of data including audio, image, text, numeric, and categorical data. We will go over some of the basic strategies for handling several of these data types.

We will have an additional lesson that focuses on processing text data. What follows are some common techniques we can use to apply rules to help bring some consistency to handling both numeric and categorical attributes.


#**Cleaning the Missing**

Many of the techniques to 'normalize' the data will fail if the transformation is done on a missing value. Missing values can truly be absent (e.g. the dreaded double comma in csv files) or marked with a special character like None, NAN, NaN, nan,null, NULL, void, 'n/a', or the empty string (i.e ''). Pandas, NumPy, and Sklearn all provide ways to help with cleaning and normalizing data.

#**A titanic example**

![](https://drive.google.com/uc?export=view&id=1T62pGhC6VkO8Ya1jJe0hvZUvooEOjtDL)

The Titanic sank on April 15th, 1912 after hitting an iceberg. It had roughly 2208 passengers and crew aboard (the exact number seems to be [unknown](http://www.icyousee.org/titanic.html).) A few good [references](https://www.historyonthenet.com/the-titanic) for some [details](https://titanicfacts.net/titanic-passengers/) show the discrepancies in the exact numbers.

The titanic dataset provides a good opportunity to work on a classic dataset that is in need of some cleaning and normalization. This lesson includes
one of the more complete datasets. There are many available.

We can load up the dataset and print out the first row so you can get a feel for the data. You can [read](http://campus.lakeforest.edu/frank/FILES/MLFfiles/Bio150/Titanic/TitanicMETA.pdf) about the meaning of each data field as well.

In [53]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

def build_titanic():
  df = pd.read_csv('titanic.csv')
  print('total rows', len(df))

  # add an extra passenger
  extra = {'name': 'Jack Dawson', 'age': 28, 'id': len(df)+1, 'gender': 'male'}
  df = df.append(extra, ignore_index=True)

  # add an extra field for using a custom transformer
  df['sid'] = df['age'].apply(lambda x: 'NA' if np.isnan(x) else "{:.0f}".format(1912-
x)).astype('string')
  df['sid'].replace('NA', np.nan, inplace=True)
  
  return df.copy()

df = build_titanic()
print(df.head(1))

total rows 2207
   id                 name gender   age class embarked        country  ticketno  fare  sibsp  parch survived   sid
0   1  Abbing, Mr. Anthony   male  42.0   3rd        S  United States    5547.0  7.11    0.0    0.0       no  1870


**Getting the missing counts**

One of the first things you should do is get a count of which attributes have missing values. You can easily do this with pandas. In the example below, we ask for all null (NaN) values and then sum them up. When you look at the result, any column with a non-zero value has missing data.

In [54]:
def show_missing(df):
  print(df.isna().sum())
  
show_missing(df)

id            0
name          0
gender        0
age           2
class         1
embarked      1
country      82
ticketno    892
fare        917
sibsp       901
parch       901
survived      1
sid           2
dtype: int64


You can see that 'age' has only 2 missing values. That is, 2 rows/instances don't have a value for the 'age' column. If a majority of your rows have missing values for a certain attribute, the best strategy is to simply not use that attribute in any of the analysis (e.g. country, ticketno, fare, sibsp, parch).

### **Imputing the missing**

One of the most common ways to deal with missing values is to interpolate (e.g. estimate or impute) the value from the values that are present.

**Pandas**

In the example below, we fill empty/missing values for the 'age' attribute with the mean for that column. You can also use the calculated median, mode or a constant as well.

In [55]:
def process_missing_age(df, debug=True):

  # mask to select the rows where age is empty
  mask = df.age.isna()

  # calculate the mean (the replacement value)
  replace_value = df.age.mean()
  
  # fill those values with the value calculated
  df['age_clean']= df[mask].age.fillna(replace_value)
  if debug:
    # print out the updates
    cols = ['id', 'name', 'age', 'age_clean']
    print(df[mask][cols].head())
    
# pass in a copy, so we keep the original
process_missing_age(df.copy())

      id                    name  age  age_clean
439  440  Gheorgheff, Mr. Stanio  NaN   30.43563
677  678     Kraeff, Mr. Theodor  NaN   30.43563


**Sklearn**

Another way to impute the missing is to use sklearn. The same 'age' column is updated to the mean for any missing value. Note how we create a new attribute to hold the 'age' column that is free of any missing values. 

Also, read the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) for different strategies you can use. Note sklearn's repeated fit and transform process is applied here (even though there is no training and fitting of models).

In [56]:
def process_missing_age_skl(df, debug=True):
  from sklearn.impute import SimpleImputer
  import numpy as np

  # np.nan is the how pandas marks missing values
  # replace with the mean
  imr = SimpleImputer(strategy="mean", missing_values=np.nan)

  # Impute values
  values = df.age.values.reshape(-1,1)
  out = imr.fit_transform(values)

  # now we assign those values to a new column
  df['im_age'] = out

  if debug:
    cols = ['id', 'name', 'age', 'im_age']
    mask = df.age.isna()
    print(df[mask][cols].head())
    print('Missing')
    print(df.isna()[cols].sum())
    
process_missing_age_skl(df.copy())

      id                    name  age    im_age
439  440  Gheorgheff, Mr. Stanio  NaN  30.43563
677  678     Kraeff, Mr. Theodor  NaN  30.43563
Missing
id        0
name      0
age       2
im_age    0
dtype: int64


A few lines of code to note. See if you understand the following from the above code block:



```
# Impute values
values = df.age.values.reshape(-1,1)
out = imr.fit_transform(values)
```

* sklearn needs its data in numpy/array form
* .values is the underlying numpy array
* .reshape(-1,1) reshapes the data to a single feature/column
* -1,1 means all rows, 1 column

It's also possible to use a more 'user-friendly' version as well:

```
# Impute values
features = ['age']
values = df[features]
out = imr.fit_transform(values)
```

In this case, we can actually fit and transform multiple columns. If you do that it's important to take caution on how you assign the result back to the pandas dataframe.

**Deleting the missing instances (rows)**

Another, perhaps drastic, technique is to simply remove any row that has a missing value for any of the attributes you need. This strategy is fine if you have plenty of data. You can also decide to delete only if an instance has multiple missing attributes.

**Predicting the missing;**

Another viable option is to use a machine learning algorithm to figure out which value should be used to replace the missing. Although we haven't discussed the K-Nearest Neighbors (KNN) algorithm (a supervised ML algorithm for both classification and regression), it's a viable solution if you want a more robust strategy than simple statistical (e.g. mean, median, mode) replacement. If there's enough time, we will have a lesson on KNN.

#**Categorical Data**

![](https://drive.google.com/uc?export=view&id=1_8d2JROR2Oa4_XMi2UIQEkdqi4YhQJnV)

For handling categorical data or data whose attribute values are labels/text, the main focus is being consistent for handling the different categories. The goal is to map each label/category to a unique number.

Many of the same rules apply for text cleaning; however, if the label is coming from a computer, process, sensor, or standardized input, much of the cleaning is already done for you. That is, you don't have to worry about different labels having the same semantic meaning. For example, if your categories are coming from a web form, you would have to deal with misspellings, etc.

###**Mapping labels to numbers**

All machine learning algorithms need a numeric representation of any text value. If you have categorical attributes whose values are strings, you will need to map these values to a number.

**Nominal Attributes**
For simple categorical data where there is no ranking order of the values, we can simply map the labels to numbers. In Pandas, it's very straight forward. You can use the .astype("category").cat.codes on any categorical data type:

```
df['g_code'] = df['gender'].astype("category").cat.codes
```
Because the process is so straight forward, we can create a utility function to help us map any column:

In [57]:
def category_to_number(df, col_name, new_name, debug=True):

  # map the categories to unique integers (starting at 0)
  df[new_name] = df[col_name].astype("category").cat.codes

  if debug:
    values = df[col_name].unique()
    print('{:d} unique values for {:s}:'.format(len(values), col_name), values)
    # show how many rows are in each group
    print(df.groupby([col_name]).count()[new_name].reset_index())
    print(df.groupby([col_name]).count()[new_name].sum())
  
  return df

# map gender to a 0/1 code
cols = ['gender', 'g_code']
df2 = category_to_number(df.copy(), *cols)  # putting * to good use
print(df2[cols][0:10]) # first 10 rows

2 unique values for gender: ['male' 'female']
   gender  g_code
0  female     489
1    male    1719
2208
   gender  g_code
0    male       1
1    male       1
2    male       1
3  female       0
4  female       0
5    male       1
6    male       1
7  female       0
8    male       1
9    male       1


**Handling missing values**

For the 'gender' column, there are no missing values. However if there are any missing values, those values get marked with a -1 by default. If you want to keep the np.nan values for missing you can simply do a replace:

```
df2['g_code'] = df2['g_code'].replace(-1, np.nan)
```

**port of embarkation**
We can use the same function to map port of embarkation, which does have some missing values. Let's take a look at how pandas handles those missing values. Be sure to understand how pandas treats the missing values

In [58]:
# map port of embarkation (C, Q, S)
# C = Cherbourg, Q = Queenstown, S = Southampton,  B = ??
cols = ['embarked', 'e_code']
df = category_to_number(df.copy(), *cols)
print(df[cols][0:10]) # first 10 rows
mask = df.embarked.isna()
print(df[mask])

5 unique values for embarked: ['S' 'C' 'B' 'Q' nan]
  embarked  e_code
0        B     197
1        C     271
2        Q     123
3        S    1616
2207
  embarked  e_code
0        S       3
1        S       3
2        S       3
3        S       3
4        S       3
5        S       3
6        C       1
7        C       1
8        C       1
9        S       3
        id         name gender   age class embarked country  ticketno  fare  sibsp  parch survived   sid  e_code
2207  2208  Jack Dawson   male  28.0   NaN      NaN     NaN       NaN   NaN    NaN    NaN      NaN  1884      -1


**Codebook WARNING**

As you may have noticed, there's not much control over which numbers will be assigned to which categories. If you needed 'Cherbourg' to be a specific numeric value (because of an outside requirement or codebook -- a document that specifies how the mappings are done), you would have to adjust the algorithm. The next section discusses, one possible solution.

**Ordinal Mapping**
When your categorical attributes have an inherent ranking (e.g. review of stars, likert scales, etc), you can define your own value map that maintains the order as well.

**“ Likert's Log**: As a word of caution, just because there's an order of 'rank' to your values, these values are still not 'numeric'.

For example, if you had a survey that ranked items using a [Strongly Agree, Agree, Neither, Disagree, Strongly Disagree] scale or you asked someone to rank an issue from 1 to 5, don't assume you can work with averages. Also, the interval between different values isn't mathematically stable. For example, the difference between 'Strongly Agree' and 'Agree' cannot be assumed to be the same as the difference between 'Disagree' and 'Strongly Disagree'. Using counts, medians, and modes is usually the best you can do.



With Pandas, you can use the map function to create custom rank values:

```
df[new_name] = df[column_name].map(kv_map)
```

Let's use that pattern to map the class attribute that categorizes both the passengers and crew. Note that all the crew gets the value 4.

In [59]:
def map_class_attribute(df, debug=True):

  # map 1st/2nd/3rd class as ordinal 1st < 2nd < 3rd + missing
  # replace nan with the value 'unknown'
  df['class'].fillna('unknown', inplace=True)
  # assign the labels values
  ord_map = {'3rd':3, '2nd':2, '1st':1,
             'engineering crew':4,
             'victualling crew':4,
             'restaurant staff':4,
             'deck crew':4, 'unknown':0}

  # apply the map to the 'class' attribute
  df['o_class'] = df['class'].map(ord_map)

  if debug:
    print(df['class'].unique().tolist())
    # show how many rows are in each group
    print(df.groupby(['class']).count()['o_class'].reset_index())

  return df
  
df_class = map_class_attribute(df.copy())
print(df_class.head())

['3rd', '2nd', '1st', 'engineering crew', 'victualling crew', 'restaurant staff', 'deck crew', 'unknown']
              class  o_class
0               1st      324
1               2nd      284
2               3rd      709
3         deck crew       66
4  engineering crew      324
5  restaurant staff       69
6           unknown        1
7  victualling crew      431
   id                            name  gender   age class embarked        country  ticketno   fare  sibsp  parch survived   sid  e_code  o_class
0   1             Abbing, Mr. Anthony    male  42.0   3rd        S  United States    5547.0   7.11    0.0    0.0       no  1870       3        3
1   2       Abbott, Mr. Eugene Joseph    male  13.0   3rd        S  United States    2673.0  20.05    0.0    2.0       no  1899       3        3
2   3     Abbott, Mr. Rossmore Edward    male  16.0   3rd        S  United States    2673.0  20.05    1.0    1.0       no  1896       3        3
3   4  Abbott, Mrs. Rhoda Mary 'Rosa'  female  39.0  

Once we have those numeric values, it's pretty easy to separate everyone into crew and passengers:

In [60]:
def print_passenger_class_stats(df):
  is_crew = df['o_class'].isin([4])

  # any of these will work
  is_pass = df['o_class'].isin([1,2,3])
  is_pass = (df['o_class'] < 4 ) & (df['o_class'] > 0)
  is_pass = ~is_crew  # will include the unknowns

  print('crew', len(df[is_crew]))
  print('pass', len(df[is_pass]))
  
# this assumes map_class_attribute is done
print_passenger_class_stats(df_class.copy())

crew 890
pass 1318


**Binary Fields**

Sometimes you may want an attribute to simply indicate a simple Yes/No, On/Off, Have/not- Have value.

For example, we can add a is_passenger attribute to the data. This field will either be 0 (False) or 1 (True) indicating if the person was a passenger on the ship (as opposed to being part of the crew.)

In [61]:
def create_binary_field(df, debug=True):
  # Binary Fields
  from sklearn.preprocessing import Binarizer
  binarizer = Binarizer(threshold=3, copy=True)  # <= 3 asssing
  column_values = binarizer.fit_transform(df.o_class.values.reshape(-1, 1))
  
  # flip the values (1 -> 0; 0 -> 1)
  df['is_passenger'] = 1 - column_values
  if debug:
    print(column_values)
    print(df.head(5))

print(create_binary_field(df_class.copy()))

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [0]]
   id                            name  gender   age class embarked        country  ticketno   fare  sibsp  parch survived   sid  e_code  o_class  is_passenger
0   1             Abbing, Mr. Anthony    male  42.0   3rd        S  United States    5547.0   7.11    0.0    0.0       no  1870       3        3             1
1   2       Abbott, Mr. Eugene Joseph    male  13.0   3rd        S  United States    2673.0  20.05    0.0    2.0       no  1899       3        3             1
2   3     Abbott, Mr. Rossmore Edward    male  16.0   3rd        S  United States    2673.0  20.05    1.0    1.0       no  1896       3        3             1
3   4  Abbott, Mrs. Rhoda Mary 'Rosa'  female  39.0   3rd        S        England    2673.0  20.05    1.0    1.0      yes  1873       3        3             1
4   5     Abelseth, Miss. Karen Marie  female  16.0   3rd        S         Norway  348125.0   7.13    0.0    0.0      yes  1896       3        3             1
None


**One Hot Encoding**

As mentioned in a previous lesson, one hot encoding simply assigns either a 1 or a 0 to an attribute if that attribute is present or not. It's an extension of using the Binarizer. The issue is that for categorical (nominal or ordinal), it's not accurate that some values will be 'higher' than other only because their mapping value was larger. As we just saw, if you map category labels, for example, to the numbers 0 through 100 (or 0.0 to 1.0), the ML algorithm may think the higher values contributes more to some correlation or calculation than the lower values. It's also useful just to mark some attributes as being present or not. This is where one hot encoding becomes useful.
<br>
<br>

In the previous section we built a simple one-hot attribute (is_passenger), this example takes an entire column of categorical values and builds separate one-hot columns for each unique value. It's simply a convenient way to create one-hot attributes. The image below shows an example of the result of one hot encoding of the embankment attribute:


![](https://drive.google.com/uc?export=view&id=1tBf_U9C70qAmBFZgcqPltDDbv5ueC1DZ)

In [62]:
def one_hot_encoding(df):
  import numpy as np
  from sklearn.preprocessing import OneHotEncoder

  onehot = OneHotEncoder(dtype=np.int, sparse=True)
  
  # fill in any missing values with 'UNK'
  df['embarked'].fillna('UKN', inplace=True)
  values = df['embarked'].values.reshape(-1, 1)
  values = onehot.fit_transform(values).toarray() # it is sparse
  labels = onehot.categories_
  
  return pd.DataFrame(values, columns=labels)

df_hot = one_hot_encoding(df.copy())
print(df_hot.tail(20))

      B  C  Q  S UKN
2188  0  0  0  1   0
2189  0  0  0  1   0
2190  0  0  0  1   0
2191  0  0  0  1   0
2192  0  0  0  1   0
2193  0  0  0  1   0
2194  0  0  0  1   0
2195  0  0  0  1   0
2196  0  0  0  1   0
2197  0  0  0  1   0
2198  1  0  0  0   0
2199  0  0  0  1   0
2200  0  0  0  1   0
2201  0  0  0  1   0
2202  1  0  0  0   0
2203  0  0  0  1   0
2204  0  0  0  1   0
2205  0  0  0  1   0
2206  0  0  0  1   0
2207  0  0  0  0   1


The panda's get_dummies method provides another way to do one-hot encoding:

```
print(pd.get_dummies(df['embarked'], dummy_na=True))
```

#**Binning/Discretizating Features**
Although not strictly for categorical data, another option is to put data into bins. Pandas provides the cut method to create custom bins:

In [63]:
# binning data
def bin_demo1(df):
# set up custom bins
  bins = [0, 3, 8, 16, 21, 35, 55, 200] 
  labels = ['infant','child','youth','young adult','adult','middle','senior'] 
  age_bins = pd.cut(df['age'], bins=bins, labels=labels, right=False) 
  df['age_cat'] = age_bins
  print(df['id age age_cat'.split()].head(10))
  return df
  
df_bin = bin_demo1(df.copy())

   id   age      age_cat
0   1  42.0       middle
1   2  13.0        youth
2   3  16.0  young adult
3   4  39.0       middle
4   5  16.0  young adult
5   6  25.0        adult
6   7  30.0        adult
7   8  28.0        adult
8   9  27.0        adult
9  10  20.0  young adult


Sklearn provides a similar preprocessing utility class for binning, named KBinsDiscretizer. It has the familiar fit and transform API. It also requires that the attribute has no missing values.

In [64]:
def bin_demo2(df):
  # uniform bins
  from sklearn.preprocessing import KBinsDiscretizer

  # this must be done first
  df['age'].fillna(df['age'].mean(), inplace=True)
  
  binner = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='uniform')
  values = binner.fit_transform(df['age'].values.reshape(-1, 1))
  df['age_cat2'] = values
  print(df['id age age_cat age_cat2'.split()].head(10))

bin_demo2(df_bin.copy())

   id   age      age_cat  age_cat2
0   1  42.0       middle       4.0
1   2  13.0        youth       1.0
2   3  16.0  young adult       1.0
3   4  39.0       middle       4.0
4   5  16.0  young adult       1.0
5   6  25.0        adult       2.0
6   7  30.0        adult       3.0
7   8  28.0        adult       3.0
8   9  27.0        adult       2.0
9  10  20.0  young adult       2.0


We can even use the same class to create one-hot encoded bins. By changing it's strategy. Be sure to [read](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) its documentation.

In [65]:
def bin_demo3(df):
  # one hot binning
  from sklearn.preprocessing import KBinsDiscretizer

  bin_count = 4
  df['age'].fillna(df['age'].mean(), inplace=True)
  binner = KBinsDiscretizer(n_bins=bin_count, encode='onehot-dense', strategy='uniform')
  values = binner.fit_transform(df['age'].values.reshape(-1, 1))
  labels = ['bin {:d}'.format(i) for i in range(1, bin_count+1)]
  
  df2 = pd.DataFrame(values, columns=labels)
  df2['age'] = df['age']
  print(df2.head(10))

bin_demo3(df.copy())

   bin 1  bin 2  bin 3  bin 4   age
0    0.0    0.0    1.0    0.0  42.0
1    1.0    0.0    0.0    0.0  13.0
2    1.0    0.0    0.0    0.0  16.0
3    0.0    0.0    1.0    0.0  39.0
4    1.0    0.0    0.0    0.0  16.0
5    0.0    1.0    0.0    0.0  25.0
6    0.0    1.0    0.0    0.0  30.0
7    0.0    1.0    0.0    0.0  28.0
8    0.0    1.0    0.0    0.0  27.0
9    0.0    1.0    0.0    0.0  20.0


#**Numeric Data**

The goal for normalizing numeric data is essentially same: you want your values to accurately represent the underlying measurement. However, there's an additional consideration called scaling (or feature scaling). The idea is that you don't want the units of one feature (i.e. column/attribute) to overshadow the units of another.

![](https://drive.google.com/uc?export=view&id=1eFOO24Rb9gU98yH43smcy3WxxOvd41Oi)

#**Feature Scaling Data**
One of the most important transformation to make is to ensure each of your numeric attributes are scaled so that one attribute's units are in the same 'scale' as others. Feature scaling standardizes the value range of features of the data.

Let's go over some vocabulary relevant for feature 'scaling':
* **Rescaling** a feature vector (think columns of data) means to add or subtract a constant and then multiply or divide by a constant, as you would do to change the units of measurement of the data, for example, to convert a temperature from Celsius to Fahrenheit.

* **Normalizing** a feature vector usually means dividing each element by the norm (L1 or L2) of the vector (||x||). However, for this lesson it will refer to a type of rescaling.

* **Standardizing** a vector means subtracting a measure of location (mean or median) and dividing by a measure of scale (e.g. standard deviation).

**Min-Max Scaling**

**(a.k.a Min-Max Normalization)**

In min-max scaling, you transform the data such that the features are within a specific range usually [0, 1].

Scaling is important for algorithms where distance between data points is important. You want to avoid attributes working together but are on different scales. For example, having one attribute measured in feet while the other is in pounds while another is in miles, will wreak havoc on the ML algorithm's calculations.

![](https://drive.google.com/uc?export=view&id=1yjhjzsYIzUuQiL_5tI3g1Rv4E8NUIkzX)


You can see how the min-max formula scales by the data range (also called peak-to-peak). Run the code below to view some of the values that are used to scale each data item:

In [66]:
import numpy as np
def fare_stats(df):
  df['fare'].fillna(df['fare'].mean(), inplace=True)

  print(df['fare'].min())
  min_fair = np.min(df['fare'])
  max_fair = np.max(df['fare'])
  print(min_fair, max_fair)
  print(np.ptp(df['fare']))
  
df = build_titanic()
fare_stats(df.copy())

total rows 2207
3.0305
3.0305 512.0607
509.0302


**Exercise**

Create a function named fare_min_max_scaled_np that uses NumPy to scale the attributes using the min-max formula given above.

* You can only use NumPy to do the calculations.
* Add the new values to an attribute named fare_mms 
* You must return the dataframe

In [67]:
def fare_min_max_scaled_np(df):
  df['fare'].fillna(df['fare'].mean(), inplace=True)
  min_fair = np.min(df['fare'])
  max_fair = np.max(df['fare'])
  range = max_fair - min_fair
  df['fare_mms'] = (df['fare']-min_fair)/range
  return df


Be sure to write your own test for fare_min_max_scaled_np function. How will you confirm that the operation succeeded?

```
def test_fare_mms(df):
  pass

test_fare_mms(df.copy())
```

In [68]:
def test_fare_mms(df):
  return print(df['fare_mms'].between(0,1).all())

df = build_titanic()
df = fare_min_max_scaled_np(df.copy())

test_fare_mms(df.copy())

total rows 2207
True


**Sklearn's Min-Max Scalar**

Sklearn also provides a simple minmax_scale function. We can use it to confirm our results as well. Below also shows how sklearn uses the fit & transform methods on its MinMaxScaler. In this case, fit would calculate the min and max first; transform then applies the formula to all the data.

In [69]:
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler

def fare_min_max_scaled(df):
  df['fare_scaled'] = minmax_scale(df['fare'])

  # fit & transform way
  scaler = MinMaxScaler()
  s_values = scaler.fit_transform(df['fare'].values.reshape(-1,1))
  df['fare_scaled2'] = s_values
  print(df['fare fare_scaled fare_scaled2'.split()].head(10))

fare_min_max_scaled(df.copy())

      fare  fare_scaled  fare_scaled2
0   7.1100     0.008014      0.008014
1  20.0500     0.033435      0.033435
2  20.0500     0.033435      0.033435
3  20.0500     0.033435      0.033435
4   7.1300     0.008054      0.008054
5   7.1300     0.008054      0.008054
6  24.0000     0.041195      0.041195
7  24.0000     0.041195      0.041195
8  18.1509     0.029704      0.029704
9   7.1806     0.008153      0.008153


**Mean Normalization Mean Centering**

Another version of Min-Max Scaling is named mean normalization. In this case, the mean is subtracted from each value (rather than the minimum value).

Note that Mean centering is the subtraction part -- and it can be done without any further scaling. The dividing by the data range provides the scaling part.

**Z-Score Scaling; Standardization**

Sometimes it's important that an attribute has certain statistical requirements. We can 'normalize' (here normalize is used in the statistical sense) the values such that the data is centered at 0 with a standard deviation of 1. This technique is called z-score scaling or just standardization or z-score normalization.

Z-score scaling is done using the following formula:

![](https://drive.google.com/uc?export=view&id=1p2OVmwJvXRr-YTkxiOEtmDu3rHLRTngr)

Here is the original feature vector, is the mean of that feature vector, and σ is its standard deviation. By subtracting the mean from the distribution, you are essentially shifting it towards left or right by amount equal to mean. By dividing by the standard deviation σ, you are changing the shape of distribution.

The new standard deviation of this standardized distribution is 1 and μ = 0.
With sklearn, you can do this transformation using the scale function or the StandardScaler class:

In [70]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

def fare_z_scaled(df):

  # simple way
  df['fare_z'] = scale(df['fare'])

  # fit & transform way
  scaler = StandardScaler()
  s_values = scaler.fit_transform(df['fare'].values.reshape(-1,1))
  df['fare_z2'] = s_values
  print(df['fare fare_z fare_z2'.split()].head(10))

fare_z_scaled(df.copy())

      fare    fare_z   fare_z2
0   7.1100 -0.658679 -0.658679
1  20.0500 -0.334534 -0.334534
2  20.0500 -0.334534 -0.334534
3  20.0500 -0.334534 -0.334534
4   7.1300 -0.658178 -0.658178
5   7.1300 -0.658178 -0.658178
6  24.0000 -0.235588 -0.235588
7  24.0000 -0.235588 -0.235588
8  18.1509 -0.382107 -0.382107
9   7.1806 -0.656911 -0.656911


#**Outlier Detection**

Having outliers in the dataset, can affect scaling. Although there are machine learning algorithms to help with outlier detection, there are a couple of simple approaches.
<br><br>

**Removing the outliers**

A very quick way to isolate the outliers is to remove those values that are over 2.5 standard deviations away from the rest of the values.

In [71]:
import numpy as np
def find_fare_outliers(df):
  df['fare'].fillna(df['fare'].mean(), inplace=True)
  data = df['fare'].values.reshape(-1,1)

  m = np.mean(data)
  s = np.std(data)
  
  # identify outliers
  cut_off = s * 3.5 # pick any number of standard deviations (usually >= 2.0)
  lower, upper = m - cut_off, m + cut_off
  # identify outliers
  outliers = [x for x in data if x < lower or x > upper]
  print("{:d} outliers: min {:.2f} max {:.2f}".format(len(outliers), np.min(outliers),
np.max(outliers)))
  
  # remove outliers
  # outliers_removed = [x for x in data if x >= lower and x <= upper]

find_fare_outliers(df.copy())

38 outliers: min 211.06 max 512.06


You can actually remove the outliers by uncommenting the last line.

**Robust Scaling**

The mean is highly sensitive to outliers. Sklearn's RobustScaler using IQR (interquartile range) to keep all values between the 25th and 75 quartile. It subtracts the column's median and divides by the interquartile range.

This scales the data using


![](https://drive.google.com/uc?export=view&id=1U06oPdGbrrt9QVjmX27vdNLaa6GNtO6c)

RobustScaler can be used when you want to reduce the effects of having many outliers. However, removing unimportant outliers (see above) should also be done.

**Confused ?**

It's not necessarily easy to know which technique to use. The best strategy is to evaluate models with the data prepared using different techniques. Sometimes, it's a combination of a few of the methods.

There are a few guidelines to help make a first approximation:
* if the distribution of the quantity is normal, then it should be standardized,
* if the distribution is not normal, the data should be normalized. This applies if the range of quantity values is large (10s, 100s, etc.) or small (0.01, 0.0001).

**Min-Max Scaler**

* Re-scales to predetermined range [0–1]
* Typical neural network algorithm require data that on a 0-1 scale.
* Doesn’t change distribution’s center (doesn’t correct skewness)
* The distribution of the feature (or any transformations of the feature ) isn’t Gaussian
* Feature falls within a bounded interval (sensitive to outliers)

**Standard Scaler**
* Shifts distribution’s mean to 0 and unit variance
* No predetermined range
* Best to use on data that is approximately normally distributed clustering, PCA (those that rely on using variance)

**Robust Scaler**
* 0 mean and unit variance
* Use of quartile ranges makes this less sensitive to (a few) outliers • No predetermined range

**Custom Cleaning**

Sometimes, it's necessary to provide cleaning beyond what a library has to offer. Sklearn's FunctionTransformer can be used in these situations. For example, the dataset has an 'sid' field which is actually a string representing the birth year of the person on board the Titanic. If you wanted to do 'math' on that field, a FunctionTransformer could be used to convert that string into a number:

The following code (you should implement it) demonstrates how to convert a string field (sid) to an integer. Once that is done, the field can be used like any valid numeric attribute:


> **Coder's Log:** You may see references to integer (int for short) or floating point (float for short). These are different ways to represent numbers. A floating point is a number with a decimal point (e.g 123.45) and an integer has no decimal place (whole numbers). Floating point numbers have more precision.

```
import numpy as np
from sklearn.preprocessing import FunctionTransformer

def string_to_float(v):
  # v is an array of values
  return v.astype(np.float)

def string_to_int(v):
  # v is an array of values
  return v.astype(np.int)

def clean_sid(df):

  # first clean any missing values
  mode = df['sid'].mode()[0]
  df['sid'].fillna(mode, inplace=True)

  # because 'sid' is pandas StringArray, reshape(-1,1) won't work
  # print(type(df['sid'].values))

  # either of these will work
  values = df['sid'].values
  # OR
  # attribute = ['sid']
  # values = df[attribute]
  
  transformer = FunctionTransformer(string_to_int)
  df['sid'] = transformer.fit_transform(values)
  return df
```

The following code cell demonstrates the custom cleaning:

In [72]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

def string_to_float(v):
  # v is an array of values
  return v.astype(np.float)

def string_to_int(v):
  # v is an array of values
  return v.astype(np.int)

def clean_sid(df):

  # first clean any missing values
  mode = df['sid'].mode()[0]
  df['sid'].fillna(mode, inplace=True)

  # because 'sid' is pandas StringArray, reshape(-1,1) won't work
  # print(type(df['sid'].values))

  # either of these will work
  values = df['sid'].values
  # OR
  # attribute = ['sid']
  # values = df[attribute]

  transformer = FunctionTransformer(string_to_int)
  df['sid'] = transformer.fit_transform(values)
  return df

In [73]:

def demo_custom_cleaning(df):
  try:
    # this will not work
    df['sid'] = df['sid'] - 1912
  except Exception as e:
    print('invalid math')

  # clean it so we can do math on it
  df_c = clean_sid(df)

  df_c['sid'] = 1912 - df_c['sid']
  print(df_c.head())
  
demo_custom_cleaning(df.copy())

invalid math
   id                            name  gender   age class embarked        country  ticketno   fare  sibsp  parch survived  sid  fare_mms
0   1             Abbing, Mr. Anthony    male  42.0   3rd        S  United States    5547.0   7.11    0.0    0.0       no   42  0.008014
1   2       Abbott, Mr. Eugene Joseph    male  13.0   3rd        S  United States    2673.0  20.05    0.0    2.0       no   13  0.033435
2   3     Abbott, Mr. Rossmore Edward    male  16.0   3rd        S  United States    2673.0  20.05    1.0    1.0       no   16  0.033435
3   4  Abbott, Mrs. Rhoda Mary 'Rosa'  female  39.0   3rd        S        England    2673.0  20.05    1.0    1.0      yes   39  0.033435
4   5     Abelseth, Miss. Karen Marie  female  16.0   3rd        S         Norway  348125.0   7.13    0.0    0.0      yes   16  0.008054


**Pandas Coercion**

Although sklearn's FunctionTransformer is incredibly flexible, for simple type conversion (string-to-int, int-to-string, etc), you can use panda's to_numeric method. If any issues happen in the coercion, np.nan will be used:

```
values = pd.to_numeric(df['xyz_column'], errors='coerce')
```

The to_numeric method will convert values to floats (real numbers). If you want the conversion to be a specific type (like integer) you can either use the downcast [flag](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html) or the astype method:

```
values = df['xyz'].astype(int) # if any issues, an exception is thrown
values = df['xyz'].astype(int, errors='ignore') # ignore those exceptions
```

**Date and Time Care**

Handling date and time attributes is a mix of working with text, categorical, and numerical. How you handle the attribute just depends on the level of granularity and its purpose. For example, if you are just marking the month of a purchase, treating dates at the 'month' level as a category is perfectly reasonable. However, if you are working with timestamps of events, then these become numeric fields that you need to work with.

Deciding to scale a date/time field is also application specific. If the year of when a car was manufactured becomes important, than that field can be treated as an integer. Another issue is that more recent years will have a higher weight (the scaled number would be closer to 1) than those that happened early on. This may or may not be what you want.

**What is Regularization -- it sounds like normalization?**

Regularization has to do with preventing overfitting. They are the techniques used to reduce the error by fitting a function appropriately on the given training set and avoid overfitting.

Regularization can be controlled via hyperparameters (those values that are given to configure the algorithm to build a model. Going into the specifics of regularization would be difficult here since it's more appropriate to discuss this when we are trying to avoid over- fitting by adjusting the loss (or objective function). You may hear about L1/L2 regularization which uses the same 'math' as L1, L2 normalization.

As a reminder, Overfitting is when the model doesn't generalize the 'pattern' being learned, but 'memorizes' it instead. Regularization attempts to prevent models from overfitting by using a hyperparameter to affecting the parameters or weights the model is learning.

#**Lesson Assignment**
**Cleaning Cars**

![](https://drive.google.com/uc?export=view&id=1A8hVea1Nx8Ezc4VvFmGx8sMT_xg8ng07)

This lesson will be using a classic dataset on automobiles. The origin of this dataset can be found [here](http://lib.stat.cmu.edu/datasets/).

However, before we can use it for any machine learning algorithms, it needs a lot of cleaning. We will guide you through all the steps that need to be done. Be sure to print out the dataset (df.head()) so you can first familiarize yourself with the data.

For all questions you can solve by using the information in this lesson and (if necessary) the official pandas. (e.g. see [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html)).


**Build the cars**

Create a function named build_cars that does the following:
* loads 'cars.csv' into a pandas dataframe 
* returns the dataframe



```
def build_cars():
   # return the pandas dataframe with cars.csv loaded
   return None
```



In [74]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

def build_cars():
  df = pd.read_csv('cars.csv')
  print('total rows', len(df))
  return df.copy()

df = build_cars()

total rows 262


**Clean Columns**

Now do the following to the dataframe

* removes the notes and id columns
* remove all spaces from column names
* replaces all empty string values with np.nan
   * see DataFrame.replace method for one possible option
* You can use the df.rename or df.columns.str (which allows you access to all the string methods).



In [75]:
def clean_columns(df):
  df = df[['mpg', ' cylinders', ' cubicinches', ' hp', ' weightlbs', ' time-to-60', ' year', ' brand']]
  df.columns = df.columns.str.replace(' ', '')
  df = df.replace(r'^\s*$', np.NaN, regex=True)
  return df

df = clean_columns(df)

##**Clean nan values**

For the following columns replace all empty/missing values with the requested value 

**clean the mpg field**

Create a function named clean_mpg
* replaces missing values with the median value

In [76]:
def clean_mpg(df):
  df['mpg'].fillna(df['mpg'].median(), inplace=True)
  return df

df = clean_mpg(df)
  

**clean the time-to-60 field** 

Create a function named clean_t60
* replaces missing values with the median value

In [77]:
def clean_t60(df):
  df['time-to-60'].fillna(df['time-to-60'].median(), inplace=True)
  return df

df = clean_t60(df)

**clean the year field**

Create a function named clean_year

* replaces missing values with the mode 
* converts the field to an integer value

In [78]:
def clean_year(df):
  mode = df['year'].mode()[0]
  df['year'].fillna(mode, inplace=True)
  values = df['year'].values
  transformer = FunctionTransformer(string_to_int)
  df['year'] = transformer.fit_transform(values)
  return df

df = clean_year(df)

**clean the cubicinches field**

Create a function named clean_ci
* replaces missing values with the mode
* converts the field to an integer value by using the FunctionTransformer

In [79]:
def clean_ci(df):
  mode = df['cubicinches'].mode()[0]
  df['cubicinches'].fillna(mode, inplace=True)
  values = df['cubicinches'].values
  transformer = FunctionTransformer(string_to_int)
  df['cubicinches'] = transformer.fit_transform(values)
  return df

df = clean_ci(df)

**clean the weightlbs field**

Create a function named clean_wlb
* replaces missing values with the mean


In [80]:
def clean_wlb(df):
  values = df['weightlbs'].values
  transformer = FunctionTransformer(string_to_float)
  df['weightlbs'] = transformer.fit_transform(values)
  df['weightlbs'].fillna(df['weightlbs'].mean(), inplace=True)
  return df

df = clean_wlb(df)

**clean the brand field**

Create a function named clean_brand
* remap this categorical field: 
* US becomes 0
* Europe becomes 1 
* Japan becomes 2
* place the new value in the field manf

In [81]:
def clean_brand(df):
  df['brand'].fillna('unknown', inplace=True)
  ord_map = {' Japan.':2, ' US.':0, ' Europe.':1}
  df['manf'] = df['brand'].map(ord_map)
  return df

df = clean_brand(df)


###**Scaling Features**

Create a function named scale_features that transforms a list of column names to [0,1] using sklearn's min-max scaling.
* return a new dataframe with the scaled columns 

For example, this code

```
df_sub = scale_features(cars_df, ['mpg', 'hp])
print(df_sub.head())
```

would generate this output (before and after)

```
    mpg   hp
0  22.0   69
1  16.0  100
2  14.0  165
3  31.9   71
4  17.0  140
        mpg        hp
0  0.327869  0.125000
1  0.163934  0.293478
2  0.109290  0.646739
3  0.598361  0.135870
4  0.191257  0.510870
```

In [82]:

def scale_features(df, feature_list):
    for column in feature_list:
        df[column] = minmax_scale(df[column])
    return df

df_sub = scale_features(df, ['mpg', 'hp'])
print(df_sub.head())

        mpg  cylinders  cubicinches        hp  weightlbs  time-to-60  year     brand  manf
0  0.327869          4           97  0.125000     2050.0           6  1985    Japan.     2
1  0.163934          6          250  0.293478     3278.0          18  1974       US.     0
2  0.109290          8          350  0.646739     4209.0          12  1972       US.     0
3  0.598361          4           89  0.135870     1925.0          14  1980   Europe.     1
4  0.191257          8          302  0.510870     3449.0          11  1971       US.     0


###**Flip Features**

For the 'time-to-60' a lower value is 'better'. We can remap this column so those cars with fast 'time-to-60' values (low numbers) are close to 1 and those with slow 'time-to-60' values (high numbers) are close to 0. We can simply invert these values.

Create a function called flip_features that takes a dataframe list of column names that inverts the values. For example, for the following code:

```
dfs = scale_features(dfc.copy(), ['time-to-60'])
print(dfs.head(5))
print(flip_features(dfs.copy(), ['time-to-60']).head(5))
```

The output would look like this:

```
   time-to-60  (OLD)
0    0.000000
1    0.631579
2    0.315789
3    0.421053
4    0.263158
   time-to-60  (Flipped)
0    1.000000
1    0.368421
2    0.684211
3    0.578947
4    0.736842
```

In [83]:
def flip_features(df, features):
  for column in features:
    df[column] = 1 - df[column]
  return df

In [84]:
dfs = scale_features(df.copy(), ['time-to-60'])

print(dfs.head(5))

print(flip_features(dfs.copy(), ['time-to-60']).head(5))

        mpg  cylinders  cubicinches        hp  weightlbs  time-to-60  year     brand  manf
0  0.327869          4           97  0.125000     2050.0    0.000000  1985    Japan.     2
1  0.163934          6          250  0.293478     3278.0    0.631579  1974       US.     0
2  0.109290          8          350  0.646739     4209.0    0.315789  1972       US.     0
3  0.598361          4           89  0.135870     1925.0    0.421053  1980   Europe.     1
4  0.191257          8          302  0.510870     3449.0    0.263158  1971       US.     0
        mpg  cylinders  cubicinches        hp  weightlbs  time-to-60  year     brand  manf
0  0.327869          4           97  0.125000     2050.0    1.000000  1985    Japan.     2
1  0.163934          6          250  0.293478     3278.0    0.368421  1974       US.     0
2  0.109290          8          350  0.646739     4209.0    0.684211  1972       US.     0
3  0.598361          4           89  0.135870     1925.0    0.578947  1980   Europe.     1

**Steps to submit your work:**


1.   Download the notebook from Moodle. It is recommended that you use Google Colab to work on it.
2.   Upload any supporting files using file upload option within Google Colab.
3.   Complete the exercises and/or assignments
4.   Download as .ipynb
5.   Name the file as "lastname_firstname_WeekNumber.ipynb"
6.   After following the above steps, submit the final file in Moodle





<h1><center>The End!</center></h1>